In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os; home = os.path.expanduser('~')
import sys
sys.path.append('../src')
sys.path.append(home + '/github/household-consumption/dicts')

import json_utils
import utils

from simpledbf import Dbf5

raw  = home + '/Dropbox/data/mexico_enigh/raw/'
interim = home + '/Dropbox/data/mexico_enigh/interim2/'
dicts = home + 'github/household-consumption/mexico_enigh/dicts/'

In [36]:
os.chdir(dicts)
income_cat = json_utils.load_json("capital_perceptions.json")
rename = json_utils.load_json("rename_capital.json")

years = ['1994', '1996']
dfs_out = []

In [37]:
income_cat

{'P016': 'income_land_landrental',
 'P017': 'income_rental_houses_buildings_other_realestate',
 'P018': 'income_interest_fixed_investments',
 'P019': 'income_interest_savings_accounts',
 'P020': 'income_interest_loans_to_third_parties',
 'P021': 'income_interest_shares_bonds',
 'P022': 'income_rental_trademarks_patents_copyrights',
 'P031': 'income_withdrawal_savings',
 'P032': 'income_from_loans_to_third_parties_not_in_household',
 'P033': 'income_from_loans_to_institutions_not_mortgage',
 'P034': 'income_coins_preciousmetals_art',
 'P035': 'income_securities_bonds',
 'P036': 'income_trademarks_patents_copyrights',
 'P037': 'income_inheritance_dowrie_legacies',
 'P038': 'income_house_land_condiminum',
 'P039': 'income_hhbusiness_machinery_animal_equipment',
 'P040': 'income_realestate_mortgage'}

In [38]:
year = '1996'
os.chdir(raw + year)

df = Dbf5('income.dbf').to_dataframe()
df.columns = [x.lower() for x in df.columns]
df = df.rename(columns = rename)
df = df.sort_values(['hhid', 'category'])
df.head(20)

,hhid,num_ren,empleo,meses,category,ing1,income,ing2,ing3,ing4,ing5,ing6
0,19960110010,01,0,070605040302,P027,150.0,550.0,150.0,200.0,200.0,200.0,200.0
1,19960110020,01,1,070605040302,P001,660.0,1500.0,600.0,600.0,180.0,180.0,780.0
2,19960110020,03,1,070605040302,P007,130.0,65.0,0.0,0.0,0.0,0.0,0.0
3,19960110030,01,1,070605040302,P007,700.0,2250.0,700.0,750.0,700.0,850.0,800.0
4,19960110030,01,1,070605040302,P032,500.0,500.0,100.0,100.0,100.0,200.0,0.0
5,19960110040,02,0,070605040302,P028,0.0,2850.0,3700.0,0.0,2000.0,0.0,0.0
6,19960110040,02,0,070605040302,P032,1000.0,500.0,0.0,0.0,0.0,0.0,0.0
7,19960110050,01,0,070605040302,P027,50.0,275.0,100.0,100.0,100.0,100.0,100.0
10,19960110060,03,1,070605040302,P001,300.0,900.0,300.0,300.0,300.0,300.0,300.0
8,19960110060,01,1,070605040302,P006,900.0,1675.0,500.0,350.0,400.0,1200.0,0.0


In [39]:
dfs = []
dfs_has = []
for year in years:
    os.chdir(raw + year)
    df = Dbf5('income.dbf').to_dataframe()
    df.columns = [x.lower() for x in df.columns]
    df = df.rename(columns = rename)
    df = df.sort_values(['hhid', 'category'])
    hhid     = np.sort(df['hhid'].unique()).astype('int')

    df['hhid'] = df['hhid'].astype(str)
    df = df[df['hhid'].apply(lambda x : x.isnumeric())]

    df['category'] = df['category'].astype('str')
    df = df[df['category'].apply(lambda x : x in income_cat.keys())]
    df['category'] = df['category'].apply(lambda x : income_cat[x])

    category = income_cat.values()

    # sum over individual expenditures within each category
    df = df.groupby(['hhid', 'category'])['income'].apply(np.sum).reset_index()

    df = df.sort_values(['hhid', 'category'])
    df['has'] = (df['income'] > 0) + 0
    df_has = df[['hhid', 'category', 'has']].pivot(index='hhid', columns='category', values='has').reset_index()

    df_has = df_has.astype(np.float)
    df_has[np.isnan(df_has)] = 0
    df_has['hhid'] = df_has['hhid'].astype(int)

    df_has.columns = ['has_' + x  if x != 'hhid' else x for x in df_has.columns]

    df = df.drop(columns='has')
    df = df.pivot(index='hhid', columns='category', values='income').reset_index()
    df['hhid'] = df['hhid'].astype(int)
    df[np.isnan(df)] = 0.0

    dfs.append(df)
    dfs_has.append(df_has)

<ipython-input-39-44109a8c0090>:27: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_has = df_has.astype(np.float)
<ipython-input-39-44109a8c0090>:27: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_has = df_has.astype(np.float)


In [40]:
df = pd.concat(dfs)
df_has = pd.concat(dfs_has)

In [41]:
for x in dfs:
    print(len(x))

2890
3594


In [42]:
has_income = [x for x in df_has.columns if 'has_income' in x]
df_has[has_income] = df_has[has_income].astype(float)
df_has

,hhid,has_income_coins_preciousmetals_art,has_income_from_loans_to_institutions_not_mortgage,has_income_from_loans_to_third_parties_not_in_household,has_income_hhbusiness_machinery_animal_equipment,has_income_house_land_condiminum,has_income_interest_fixed_investments,has_income_interest_loans_to_third_parties,has_income_interest_savings_accounts,has_income_interest_shares_bonds,has_income_land_landrental,has_income_realestate_mortgage,has_income_rental_houses_buildings_other_realestate,has_income_rental_trademarks_patents_copyrights,has_income_securities_bonds,has_income_withdrawal_savings
0,19940110060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,19940110070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,19940110090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,19940110100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,19940110190,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589,19963390980,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3590,19963390990,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3591,19963399040,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3592,19963399060,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
df

category,hhid,income_coins_preciousmetals_art,income_from_loans_to_institutions_not_mortgage,income_from_loans_to_third_parties_not_in_household,income_hhbusiness_machinery_animal_equipment,income_house_land_condiminum,income_interest_fixed_investments,income_interest_loans_to_third_parties,income_interest_savings_accounts,income_interest_shares_bonds,income_land_landrental,income_realestate_mortgage,income_rental_houses_buildings_other_realestate,income_rental_trademarks_patents_copyrights,income_securities_bonds,income_withdrawal_savings
0,19940110060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0
1,19940110070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0
2,19940110090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,950.0
3,19940110100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5400.0
4,19940110190,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589,19963390980,400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3590,19963390990,0.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3591,19963399040,50.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3592,19963399060,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
df = df.merge(df_has, on='hhid')

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6484 entries, 0 to 6483
Data columns (total 31 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   hhid                                                     6484 non-null   int64  
 1   income_coins_preciousmetals_art                          6484 non-null   float64
 2   income_from_loans_to_institutions_not_mortgage           6484 non-null   float64
 3   income_from_loans_to_third_parties_not_in_household      6484 non-null   float64
 4   income_hhbusiness_machinery_animal_equipment             6484 non-null   float64
 5   income_house_land_condiminum                             6484 non-null   float64
 6   income_interest_fixed_investments                        6484 non-null   float64
 7   income_interest_loans_to_third_parties                   6484 non-null   float64
 8   income_interest_savings_acco

In [46]:
os.chdir(interim)
df.to_csv('capital_income.csv', index=False)